In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("amazon_laptop_prices_v01.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446 entries, 0 to 4445
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brand                 4446 non-null   object 
 1   model                 3282 non-null   object 
 2   screen_size           4417 non-null   object 
 3   color                 3867 non-null   object 
 4   harddisk              3870 non-null   object 
 5   cpu                   4346 non-null   object 
 6   ram                   4385 non-null   object 
 7   OS                    4420 non-null   object 
 8   special_features      2054 non-null   object 
 9   graphics              4381 non-null   object 
 10  graphics_coprocessor  2574 non-null   object 
 11  cpu_speed             1520 non-null   object 
 12  rating                2174 non-null   float64
 13  price                 4441 non-null   object 
dtypes: float64(1), object(13)
memory usage: 486.4+ KB


In [4]:
df.describe()

,rating
count,2174.000000
mean,4.086891
std,1.165337
min,1.000000
25%,4.000000
50%,4.500000
75%,5.000000
max,5.000000


In [5]:
df.isnull().sum()

brand                      0
model                   1164
screen_size               29
color                    579
harddisk                 576
cpu                      100
ram                       61
OS                        26
special_features        2392
graphics                  65
graphics_coprocessor    1872
cpu_speed               2926
rating                  2272
price                      5
dtype: int64

In [6]:
df

,brand,model,screen_size,color,harddisk,cpu,ram,OS,special_features,graphics,graphics_coprocessor,cpu_speed,rating,price
0,ROKC,NaN,14 Inches,Blue,1000 GB,Intel Core i7,8 GB,Windows 11,NaN,Integrated,Intel,1.2 GHz,NaN,$589.99
1,HP,NaN,15.6 Inches,Silver,1000 GB,Intel Core i5,64 GB,Windows 11 Pro,Backlit Keyboard,Integrated,Intel,NaN,4.5,$999.99
2,MSI,Vector GP66 12UGS-267,15.66 Inches,Core Black,NaN,Intel Core i9,32 GB,Windows 11 Home,NaN,Dedicated,NaN,1.8 GHz,5.0,"$1,599.00"
3,Apple,MacBook Air,13.3 Inches,Silver,256 GB,Unknown,8 GB,Mac OS,Backlit Keyboard,Integrated,NaN,NaN,4.8,$689.99
4,Apple,MacBook Air,15.3 Inches,Midnight,256 GB,Unknown,8 GB,Mac OS,NaN,Integrated,NaN,NaN,4.8,"$1,144.48"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4441,MSI,Stealth GS66 12UGS-025,15.6 Inches,Core Black,NaN,Intel Core i9,32 GB,Windows 11 Home,NaN,Dedicated,NVIDIA GeForce RTX 3070,1.8 GHz,1.0,"$1,699.00"
4442,ROKC,NaN,14 Inches,Blue,1000 GB,Intel Core i7,8 GB,Windows 11,NaN,Integrated,Intel,1.2 GHz,NaN,$589.99
4443,Dell,Inspiron 3511 Laptop,15.6 Inches,Black,512 GB,Core i3,8 GB,Windows 10 Home,Wifi & Bluetooth,Integrated,NaN,NaN,NaN,$591.99
4444,Dell,Latitude 3420 Laptop,14 Inches,NaN,512 GB,Core i5,8 GB,Windows 10 Pro,NaN,Integrated,Intel Integrated Graphics,0.9 GHz,NaN,"$1,771.99"


In [20]:
df=df.dropna(subset=['model'])
df=df[df['model'].str.strip().astype(bool)]
df['harddisk']=df['harddisk'].fillna('512 GB')

In [23]:
import pandas as pd
import numpy as np
import re
print(f"Original shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Missing values:\n{df.isnull().sum()}")
print(f"Duplicates: {df.duplicated().sum()}")

# Clean column names
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

# Remove duplicates
df.drop_duplicates(inplace=True)

# Remove completely empty rows
df.dropna(how='all', inplace=True)

# Clean Price column
price_col = [col for col in df.columns if 'price' in col][0]
df[price_col] = df[price_col].astype(str).str.replace('[$,₹€£¥]', '', regex=True)
df[price_col] = df[price_col].str.replace(',', '')
df[price_col] = pd.to_numeric(df[price_col], errors='coerce')

# Remove extreme price outliers (< 100 or > 10000)
df = df[(df[price_col] >= 100) & (df[price_col] <= 10000)]

# Clean CPU column
cpu_col = [col for col in df.columns if 'cpu' in col or 'processor' in col][0]

# Extract CPU brand
df['cpu_brand'] = df[cpu_col].astype(str).str.lower()
df.loc[df['cpu_brand'].str.contains('intel', na=False), 'cpu_brand'] = 'Intel'
df.loc[df['cpu_brand'].str.contains('amd', na=False), 'cpu_brand'] = 'AMD'
df.loc[df['cpu_brand'].str.contains('apple|m1|m2', na=False), 'cpu_brand'] = 'Apple'
df.loc[~df['cpu_brand'].isin(['Intel', 'AMD', 'Apple']), 'cpu_brand'] = 'Other'

# Extract CPU type
df['cpu_type'] = 'Other'
cpu_str = df[cpu_col].astype(str).str.lower()
df.loc[cpu_str.str.contains('i9', na=False), 'cpu_type'] = 'i9'
df.loc[cpu_str.str.contains('i7', na=False), 'cpu_type'] = 'i7'
df.loc[cpu_str.str.contains('i5', na=False), 'cpu_type'] = 'i5'
df.loc[cpu_str.str.contains('i3', na=False), 'cpu_type'] = 'i3'
df.loc[cpu_str.str.contains('celeron', na=False), 'cpu_type'] = 'Celeron'
df.loc[cpu_str.str.contains('pentium', na=False), 'cpu_type'] = 'Pentium'
df.loc[cpu_str.str.contains('ryzen 9', na=False), 'cpu_type'] = 'Ryzen 9'
df.loc[cpu_str.str.contains('ryzen 7', na=False), 'cpu_type'] = 'Ryzen 7'
df.loc[cpu_str.str.contains('ryzen 5', na=False), 'cpu_type'] = 'Ryzen 5'
df.loc[cpu_str.str.contains('ryzen 3', na=False), 'cpu_type'] = 'Ryzen 3'
df.loc[cpu_str.str.contains('m1', na=False), 'cpu_type'] = 'M1'
df.loc[cpu_str.str.contains('m2', na=False), 'cpu_type'] = 'M2'

# Extract CPU speed
df['cpu_speed_ghz'] = df[cpu_col].astype(str).str.extract(r'(\d+\.?\d*)\s*ghz', flags=re.IGNORECASE)[0]
df['cpu_speed_ghz'] = pd.to_numeric(df['cpu_speed_ghz'], errors='coerce').fillna(0)

# Clean RAM column
ram_col = [col for col in df.columns if 'ram' in col or 'memory' in col]
if ram_col:
    ram_col = ram_col[0]
    df['ram_gb'] = df[ram_col].astype(str).str.extract(r'(\d+)')[0]
    df['ram_gb'] = pd.to_numeric(df['ram_gb'], errors='coerce').fillna(0)

# Clean Storage column
storage_cols = [col for col in df.columns if any(term in col.lower() for term in ['storage', 'disk', 'ssd', 'hdd'])]
if storage_cols:
    storage_col = storage_cols[0]
    df['storage_size_gb'] = df[storage_col].astype(str).str.extract(r'(\d+)')[0]
    df['storage_size_gb'] = pd.to_numeric(df['storage_size_gb'], errors='coerce').fillna(0)
    
    storage_str = df[storage_col].astype(str).str.lower()
    df['storage_type'] = 'Unknown'
    df.loc[storage_str.str.contains('ssd', na=False), 'storage_type'] = 'SSD'
    df.loc[storage_str.str.contains('hdd', na=False), 'storage_type'] = 'HDD'
    df.loc[storage_str.str.contains('emmc', na=False), 'storage_type'] = 'eMMC'

# Clean Screen size
screen_cols = [col for col in df.columns if 'screen' in col.lower() or 'display' in col.lower()]
for col in screen_cols:
    if 'size' in col.lower():
        df['screen_size_inches'] = df[col].astype(str).str.extract(r'(\d+\.?\d*)')[0]
        df['screen_size_inches'] = pd.to_numeric(df['screen_size_inches'], errors='coerce').fillna(0)
        break

# Clean categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    if col not in [cpu_col, 'cpu_brand', 'cpu_type', 'storage_type']:
        df[col] = df[col].astype(str).str.strip().str.title()
        df[col] = df[col].replace(['Nan', 'None', 'nan', ''], 'Unknown')

# Handle missing values
# Numerical columns - fill with median
numerical_cols = df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Categorical columns - fill with mode or 'Unknown'
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    mode_val = df[col].mode()
    if len(mode_val) > 0:
        df[col].fillna(mode_val[0], inplace=True)
    else:
        df[col].fillna('Unknown', inplace=True)

# Create derived features
if 'ram_gb' in df.columns:
    df['price_per_gb_ram'] = df[price_col] / (df['ram_gb'] + 1)

if 'storage_type' in df.columns and 'storage_size_gb' in df.columns:
    storage_multiplier = {'SSD': 1.5, 'HDD': 1.0, 'eMMC': 0.8, 'Unknown': 1.0}
    df['storage_efficiency'] = df['storage_size_gb'] * df['storage_type'].map(storage_multiplier)

# Performance tier
if 'cpu_type' in df.columns and 'ram_gb' in df.columns:
    cpu_scores = {'i9': 5, 'Ryzen 9': 5, 'i7': 4, 'Ryzen 7': 4, 'M2': 4,
                  'i5': 3, 'Ryzen 5': 3, 'M1': 3, 'i3': 2, 'Ryzen 3': 2,
                  'Celeron': 1, 'Pentium': 1, 'Other': 2}
    
    df['cpu_score'] = df['cpu_type'].map(cpu_scores).fillna(2)
    df['ram_score'] = (df['ram_gb'] / 4).clip(upper=4)
    df['total_score'] = df['cpu_score'] + df['ram_score']
    
    df['performance_tier'] = 'Entry-Level'
    df.loc[df['total_score'] >= 4, 'performance_tier'] = 'Budget'
    df.loc[df['total_score'] >= 6, 'performance_tier'] = 'Mid-Range'
    df.loc[df['total_score'] >= 8, 'performance_tier'] = 'High-End'
    
    df.drop(['cpu_score', 'ram_score', 'total_score'], axis=1, inplace=True)
df['color']=df['color'].replace['Unknown','Black']
# Final validation
print(f"\nFinal shape: {df.shape}")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Duplicates: {df.duplicated().sum()}")
print(f"\nNew columns created:")
new_cols = ['cpu_brand', 'cpu_type', 'cpu_speed_ghz', 'ram_gb', 'storage_size_gb', 
           'storage_type', 'screen_size_inches', 'price_per_gb_ram', 'storage_efficiency', 
           'performance_tier']
for col in new_cols:
    if col in df.columns:
        print(f"- {col}")

print(f"\nPrice statistics:")
print(df[price_col].describe())

print(f"\nPerformance tier distribution:")
if 'performance_tier' in df.columns:
    print(df['performance_tier'].value_counts())

# Save cleaned dataset
df.to_csv('laptop_prices_cleaned.csv', index=False)
print(f"\nCleaned dataset saved to: laptop_prices_cleaned.csv")

Original shape: (2482, 24)
Columns: ['brand', 'model', 'screen_size', 'color', 'harddisk', 'cpu', 'ram', 'os', 'special_features', 'graphics', 'graphics_coprocessor', 'cpu_speed', 'rating', 'price', 'cpu_brand', 'cpu_type', 'cpu_speed_ghz', 'ram_gb', 'storage_size_gb', 'storage_type', 'screen_size_inches', 'price_per_gb_ram', 'storage_efficiency', 'performance_tier']
Missing values:
brand                   0
model                   0
screen_size             0
color                   0
harddisk                0
cpu                     0
ram                     0
os                      0
special_features        0
graphics                0
graphics_coprocessor    0
cpu_speed               0
rating                  0
price                   0
cpu_brand               0
cpu_type                0
cpu_speed_ghz           0
ram_gb                  0
storage_size_gb         0
storage_type            0
screen_size_inches      0
price_per_gb_ram        0
storage_efficiency      0
performance_tier

C:\Users\HOME\AppData\Local\Temp\ipykernel_8564\3730777478.py:96: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\HOME\AppData\Local\Temp\ipykernel_8564\3730777478.py:96: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

TypeError: 'method' object is not subscriptable